1- Importing requests it allows you to send HTTP requests, without the need for manual labor. There's no need to manually add query strings to your URLs.

2- Importing getpass for a secure password prompt

3- Json is the exclusive format GitHub API sends data.

In [4]:
import requests
#from getpass import getpass
import json
from getpass import getpass
import pandas as pd
from github import Github
import networkx as nx
import numpy as np
import ast

Acquiring OAuth Token.. 

Quick reminder: OAuth is an open standard for access delegation, commonly used as a way for Internet users to grant websites or applications access to their information on other websites but without giving them the passwords.

In [5]:
token='9c2b278ca2caebabdbc1e6463ad8ef8c6e00e412'

In [6]:
username = 'YoussefEssDS' #put ur Github user name here! 
password = 'Seek greatness1' #put ur Github password name here! 
#Credentials are transmitted over a secure SSL connection
url = 'https://api.github.com/authorizations'
note = 'Aquiring_token'
post_data = {'scopes':['repo'],'note': note }
response = requests.post(url,auth = (username, password),data = json.dumps(post_data),) # Mohim this response needs input in this format dunno why!
print("API response:", response.text)
#print("Your OAuth token is", response.json()['token'])


API response: {"message":"Validation Failed","errors":[{"resource":"OauthAccess","code":"already_exists","field":"note"}],"documentation_url":"https://developer.github.com/v3/oauth_authorizations/#create-a-new-authorization"}


For now we'll try to list a user's followers.

As you can see we choosed the Tensorflow repository as a target for our study.

In [7]:
repo_user='tensorflow'
repo='tensorflow'

We will extract monthly data for contribution to the tensorflow repository between 01/2019 & 06/2019.

GitHub identifies contributors by author email address. This endpoint groups contribution counts by GitHub user, which includes all associated email addresses. To improve performance, only the first 500 author email addresses in the repository link to GitHub users. The rest will appear as anonymous contributors without associated GitHub user information.

In [8]:
from_initial_followship_dict={}
to_initial_followship_dict={}
client = Github(token,per_page=100)
user = client.get_user(repo_user)
repo = user.get_repo(repo)

In [9]:
contributors1 = [ s for s in repo.get_contributors()]
type(contributors1[1])

github.NamedUser.NamedUser

In [ ]:
contributors1 = [ s for s in repo.get_contributors() if  ]
print("Number of contributors",len(contributors))
for user in contributors:
    from_initial_followship_dict[user.login]=[follower for follower in user.get_followers()]

In [14]:
for user in contributors:
    to_initial_followship_dict[user.login]=[follower for follower in user.get_following()]

In [20]:
import numpy as np
l=np.ndarray(shape=(0,3),dtype=str)
l=np.append(l,[[4,4,4]],axis=0)
l=np.append(l,[[4,5,5]],axis=0)
l=np.append(l,[[4,4,0]],axis=0)
l=np.append(l,[[4,4,0]],axis=0)
l=np.append(l,[[4,4,1]],axis=0)
l=np.append(l,[[4,4,0]],axis=0)
l


pd.DataFrame(data=l,columns=['To','From','To is contributor?'])
pd.DataFrame(data=l).drop_duplicates()

,0,1,2
0,4,4,4
1,4,5,5
2,4,4,0
4,4,4,1


In [21]:
l=np.ndarray(shape=(0,3),dtype=str)

#Followers of contributors:
for contributor in from_initial_followship_dict.keys():
    for follower in from_initial_followship_dict[contributor]:
        l=np.append(l,[[contributor,follower.login,"contributer"]],axis=0)

#The followed by contributors:
for contributor in to_initial_followship_dict.keys():
    for followed in to_initial_followship_dict[contributor]:
        if followed in contributors:
            l=np.append(l,[[followed.login,contributor,"contributer"]],axis=0)
        else:
            l=np.append(l,[[followed.login,contributor,"non-contributer"]],axis=0)



In [22]:
followship_data_frame=pd.DataFrame(data=l,columns=['To','From','To is contributor?'])
followship_data_frame.drop_duplicates()

,To,From,To is contributor?
0,tensorflower-gardener,qiaohaijun,contributer
1,tensorflower-gardener,lukemetz,contributer
2,tensorflower-gardener,chrisber,contributer
3,tensorflower-gardener,jhyume,contributer
4,tensorflower-gardener,waitingkuo,contributer
5,tensorflower-gardener,zhenglu0,contributer
6,tensorflower-gardener,lishali,contributer
7,tensorflower-gardener,LJ001,contributer
8,tensorflower-gardener,Darwin2011,contributer
9,tensorflower-gardener,linzgood,contributer


Let's save the dataframe in a csv file to avoid extracting everytime.

In [23]:
pd.DataFrame.to_csv(followship_data_frame,'MainDF.csv')

In [2]:
followship_data_frame=pd.read_csv('MainDF.csv')

In [3]:
followship_data_frame

,Unnamed: 0,To,From,To is contributor?
0,0,tensorflower-gardener,qiaohaijun,contributer
1,1,tensorflower-gardener,lukemetz,contributer
2,2,tensorflower-gardener,chrisber,contributer
3,3,tensorflower-gardener,jhyume,contributer
4,4,tensorflower-gardener,waitingkuo,contributer
5,5,tensorflower-gardener,zhenglu0,contributer
6,6,tensorflower-gardener,lishali,contributer
7,7,tensorflower-gardener,LJ001,contributer
8,8,tensorflower-gardener,Darwin2011,contributer
9,9,tensorflower-gardener,linzgood,contributer
